In [1]:
import json
import pandas as pd
from pathlib import Path

In [2]:
paths_to_folds_results = {
    "fold_0": "resources/default-normalization/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_0/summary.json",
    "fold_1": "resources/default-normalization/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_1/summary.json",    
    "fold_2": "resources/default-normalization/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_2/summary.json",
    "fold_3": "resources/default-normalization/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_3/summary.json",
    "fold_4": "resources/default-normalization/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_4/summary.json",
}

In [5]:
with open(paths_to_folds_results['fold_0'], 'r') as file:
    results = json.load(file)
print(results.keys())

dict_keys(['foreground_mean', 'mean', 'metric_per_case'])


In [13]:
print(type(results['metric_per_case']))
metrics_per_case_df = pd.DataFrame(results['metric_per_case'])
display(metrics_per_case_df)
new_metrics_per_case = [
    {
        "prediction_file": item["prediction_file"],
        "reference_file": item["reference_file"],
        "class_label": class_,
        **metrics
    }
    for item in results['metric_per_case']
    for class_, metrics in item["metrics"].items()
]

<class 'list'>


,metrics,prediction_file,reference_file
0,"{'1': {'Dice': 0.7248431386643938, 'FN': 67111...",/media/rrojas/data3/nnUNet/results/trained_mod...,/media/rrojas/data3/nnUNet/data/preprocessed/D...
1,"{'1': {'Dice': 0.8053104256149941, 'FN': 24600...",/media/rrojas/data3/nnUNet/results/trained_mod...,/media/rrojas/data3/nnUNet/data/preprocessed/D...
2,"{'1': {'Dice': 0.0, 'FN': 0, 'FP': 1368, 'IoU'...",/media/rrojas/data3/nnUNet/results/trained_mod...,/media/rrojas/data3/nnUNet/data/preprocessed/D...
3,"{'1': {'Dice': 0.8190664920910152, 'FN': 3758,...",/media/rrojas/data3/nnUNet/results/trained_mod...,/media/rrojas/data3/nnUNet/data/preprocessed/D...
4,"{'1': {'Dice': 0.7980769230769231, 'FN': 3602,...",/media/rrojas/data3/nnUNet/results/trained_mod...,/media/rrojas/data3/nnUNet/data/preprocessed/D...


In [14]:
new_metrics_per_case[0]

{'prediction_file': '/media/rrojas/data3/nnUNet/results/trained_models/Dataset513_HCUCH_Lesions/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_0/validation/1.3.12.2.1107.5.1.4.83504.30000019103012204275800008668.nii.gz',
 'reference_file': '/media/rrojas/data3/nnUNet/data/preprocessed/Dataset513_HCUCH_Lesions/gt_segmentations/1.3.12.2.1107.5.1.4.83504.30000019103012204275800008668.nii.gz',
 'class_label': '1',
 'Dice': 0.7248431386643938,
 'FN': 67111,
 'FP': 40067,
 'IoU': 0.5684344888402115,
 'TN': 75511269,
 'TP': 141169,
 'n_pred': 181236,
 'n_ref': 208280}